In [1]:
original_price = [1, 5, 8, 9, 10, 17, 17, 20, 24, 30, 35]

In [2]:
# defaultdict 和 普通的 dict 的区别：当没有这个key时，普通的dict 会报错
# 而 defaultdict 则不会
from collections import defaultdict

In [3]:
price = defaultdict(int)

In [4]:
for i,p in enumerate(original_price):
    price[i+1] = p

In [5]:
price[11]

35

## Get the max splitting by enumerate

In [7]:
price

defaultdict(int,
            {1: 1,
             2: 5,
             3: 8,
             4: 9,
             5: 10,
             6: 17,
             7: 17,
             8: 20,
             9: 24,
             10: 30,
             11: 35})

In [8]:
# 递归：你在写一个递归函数的时候，要先假设这个函数能实现你要的结果
# 比如这里，你要先假设 r(n) 能返回最优解，
# max(price[10], r(1)+r(9), r(2)+r(8), ...... r(9)+r(1)
# 对于 r(1) + r(9)
# r(1) ==> return max(price[1] + [r(1) + r(0)])
# r(9) ==> return max(price[9] + [r(1) + r[8], r(2) + r[7]...])

def r(n):
    return max(
        [price[n]] + [r(i) + r(n-i) for i in range(1,n)]
    )

In [9]:
r(10)

30

In [11]:
# 会有大量重复的计算，所以需要较长时间
r(15)

45

In [12]:
import time

In [13]:
#@get_time
def fibonacci(n):
    if n  <= 2:
        return 1
    else:
        return fibonacci(n-1) + fibonacci(n-2)

In [14]:
start = time.time()
print(fibonacci(34))
end = time.time()
print(end-start)

5702887
1.442162275314331


In [19]:
mem = defaultdict()
def fibonacci_op(n):
    if n in mem:
        return mem[n]
    else: 
        if n <= 2:
            mem[n] = 1
            return 1
        else:
            result = fibonacci_op(n-1) + fibonacci_op(n-2)
            mem[n] = result
            return result

In [242]:
start = time.time()
print(fibonacci_op(64))
end = time.time()
print(end-start)

14662949395604
0.0003120899200439453


# Analysis: How to optimize

## A Simpler Problem

### Decorator

In [21]:
def get_time(func):
    def wrapper(*args):
        start = time.time()
        func(*args)
        end = time.time()
        print('used time : {}'.format(end-start))
    return wrapper

In [22]:
def f1(func):
    def wrapper(*args,**kwargs):
        print('Started')
        func(*args,**kwargs)
        print('Ended')
    return wrapper

In [23]:
def f():
    print('HELLO')

In [24]:
f = f1(f)

In [25]:
print(f.__name__)

wrapper


In [26]:
@f1
def g(a):
    print(a)

In [27]:
g('hello')

Started
hello
Ended


In [28]:
def k(*arg,**kwargs):
    print(kwargs)

In [29]:
k(6,b=5)

{'b': 5}


In [30]:
from functools import wraps

In [33]:
# Python装饰器（decorator）在实现的时候，被装饰后的函数其实已经是另外一个函数了（函数名等函数属性会发生改变），
# 为了不影响，Python的functools包中提供了一个叫wraps的decorator来消除这样的副作用。
# 写一个decorator的时候，最好在实现之前加上functools的wrap，它能保留原有函数的名称和docstring。
def memo(f):
    memo.already_computed = {}
    @wraps(f)
    def _wrap(arg):
        if arg in memo.already_computed:
            result = memo.already_computed[arg]
        else:
            result = f(arg)
            memo.already_computed[arg] = result
        return result
    return _wrap

# We use this method to solve Cut Rod probelm¶

In [34]:
solution = {}

In [39]:
@memo
def r(n):
    """
    Args: n is the iron length
    Return: the max revenue 
    """
    max_price, max_split = max(
        [(price[n], 0)] + [(r(i) + r(n-i), i) for i in range(1, n)], key=lambda x: x[0]
    )

    solution[n] = (n - max_split, max_split)
    
    return max_price

In [40]:
r(20)

60

In [37]:
price

defaultdict(int,
            {1: 1,
             2: 5,
             3: 8,
             4: 9,
             5: 10,
             6: 17,
             7: 17,
             8: 20,
             9: 24,
             10: 30,
             11: 35,
             15: 0,
             14: 0,
             13: 0,
             12: 0,
             20: 0,
             19: 0,
             18: 0,
             17: 0,
             16: 0})

In [38]:
solution

{1: (1, 0),
 2: (2, 0),
 3: (3, 0),
 4: (2, 2),
 5: (3, 2),
 6: (6, 0),
 7: (6, 1),
 8: (6, 2),
 9: (6, 3),
 10: (10, 0),
 11: (11, 0),
 12: (11, 1),
 13: (11, 2),
 14: (11, 3),
 15: (13, 2),
 16: (14, 2),
 17: (11, 6),
 18: (17, 1),
 19: (17, 2),
 20: (17, 3)}

# How do we parse solution?¶

In [101]:
def parse_solution(n):
    left_split, right_split = solution[n]
    
    if right_split == 0: return [left_split]
    
    return parse_solution(left_split) + parse_solution(right_split)

In [104]:
r(24)

75

In [108]:
parse_solution(20)

[11, 6, 3]

# Edit Distance

In [41]:
solution = {}

In [42]:
from functools import lru_cache

In [43]:
@lru_cache(maxsize=2**10)
def edit_distance(string1, string2):
    
    if len(string1) == 0: return len(string2)
    if len(string2) == 0: return len(string1)
    
    tail_s1 = string1[-1] # s1='word' s2='work' tail_s1='d' tail_s2='k'
    tail_s2 = string2[-1] # 'wor'+'d' 'wor'+'k'
     
    candidates = [
        (edit_distance(string1[:-1], string2) + 1, 'DEL {}'.format(tail_s1)),  
        # string 1 delete tail
        (edit_distance(string1, string2[:-1]) + 1, 'ADD {}'.format(tail_s2)),  
        # string 1 add tail of string2
    ]
    
    if tail_s1 == tail_s2:
        both_forward = (edit_distance(string1[:-1], string2[:-1]) + 0, '')
    else:
        both_forward = (edit_distance(string1[:-1], string2[:-1]) + 1, 'SUB {} => {}'.format(tail_s1, tail_s2))

    candidates.append(both_forward)
    
    min_distance, operation = min(candidates, key=lambda x: x[0])
    
    solution[(string1, string2)] = operation 
    
    return min_distance

In [44]:
edit_distance('ABCDECG','ABCCEF')

3

In [45]:
solution

{('A', 'A'): '',
 ('A', 'AB'): 'ADD B',
 ('A', 'ABC'): 'ADD C',
 ('A', 'ABCC'): 'ADD C',
 ('A', 'ABCCE'): 'ADD E',
 ('A', 'ABCCEF'): 'ADD F',
 ('AB', 'A'): 'DEL B',
 ('AB', 'AB'): '',
 ('AB', 'ABC'): 'ADD C',
 ('AB', 'ABCC'): 'ADD C',
 ('AB', 'ABCCE'): 'ADD E',
 ('AB', 'ABCCEF'): 'ADD F',
 ('ABC', 'A'): 'DEL C',
 ('ABC', 'AB'): 'DEL C',
 ('ABC', 'ABC'): '',
 ('ABC', 'ABCC'): 'ADD C',
 ('ABC', 'ABCCE'): 'ADD E',
 ('ABC', 'ABCCEF'): 'ADD F',
 ('ABCD', 'A'): 'DEL D',
 ('ABCD', 'AB'): 'DEL D',
 ('ABCD', 'ABC'): 'DEL D',
 ('ABCD', 'ABCC'): 'SUB D => C',
 ('ABCD', 'ABCCE'): 'ADD E',
 ('ABCD', 'ABCCEF'): 'ADD F',
 ('ABCDE', 'A'): 'DEL E',
 ('ABCDE', 'AB'): 'DEL E',
 ('ABCDE', 'ABC'): 'DEL E',
 ('ABCDE', 'ABCC'): 'DEL E',
 ('ABCDE', 'ABCCE'): '',
 ('ABCDE', 'ABCCEF'): 'ADD F',
 ('ABCDEC', 'A'): 'DEL C',
 ('ABCDEC', 'AB'): 'DEL C',
 ('ABCDEC', 'ABC'): 'DEL C',
 ('ABCDEC', 'ABCC'): '',
 ('ABCDEC', 'ABCCE'): 'DEL C',
 ('ABCDEC', 'ABCCEF'): 'SUB C => F',
 ('ABCDECG', 'A'): 'DEL G',
 ('ABCDECG', 'A

## Todo: Parse Solution is our homework

In [77]:
def distance_parse_solution(string1, string2):  
    if string1 == string2:
       return detail_solution 

    result = solution[(string1,string2)]
#     print(result)
    if 'DEL' in result:
#         print('DEL')
        detail_solution.append(f"{string1} {result}")
        string1 = string1[:-1]
    elif 'ADD' in result:
#         print('ADD')
        detail_solution.append(f"{string1} {result}")
        string2 = string2[:-1]
    elif 'SUB' in result:
#         print('SUB')
        detail_solution.append(f"{string1} {result}")
        string1 = string1[:-1]
        string2 = string2[:-1]
    else:
        print("else")
        string1 = string1[:-1]
        string2 = string2[:-1]
#     print(detail_solution)
#     print(string1)
#     print(string2)
    # 为什么递归的这个地方要加 return ？？？
    return distance_parse_solution(string1, string2)

In [78]:
detail_solution = []
distance_parse_solution('ABCD', 'ABCCEF')    

['ABCD ADD F', 'ABCD ADD E', 'ABCD SUB D => C']

In [79]:
detail_solution = []
distance_parse_solution('A', 'ABCCEF')    

['A ADD F', 'A ADD E', 'A ADD C', 'A ADD C', 'A ADD B']

# Problem Case 3: Pinyin Auto Correction Problem

In [28]:
chinese_dataset = '/Users/huazai/Desktop/学习/NLP/第1节/article_9k.txt'

In [29]:
CHINESE_CHARATERS = open(chinese_dataset).read()

In [30]:
CHINESE_CHARATERS[:40]

'此外自本周6月12日起除小米手机6等15款机型外其余机型已暂停更新发布含开发版体'

In [31]:
import pinyin

In [32]:
pinyin.get('你好，中国',format='strip',delimiter=' ')

'ni hao ， zhong guo'

In [33]:
def chinese_to_pinyin(character):
    return pinyin.get(character, format='strip', delimiter=' ')

In [34]:
CHINESE_CHARATERS_COPYS = chinese_to_pinyin(CHINESE_CHARATERS)

In [35]:
len(CHINESE_CHARATERS_COPYS)

129433034

In [36]:
import re

In [37]:
def tokens(text):
    "List all the pinyin characters"
    return re.findall('[a-z]+',text.lower())

In [38]:
CHINESE_CHARATERS_COPYS[:100]

'ci wai zi ben zhou 6 yue 1 2 ri qi chu xiao mi shou ji 6 deng 1 5 kuan ji xing wai qi yu ji xing yi '

In [40]:
from collections import Counter, defaultdict

In [41]:
# 拼音：次数
PINYIN_COUNT = Counter(tokens(CHINESE_CHARATERS_COPYS))

In [47]:
def correct(word):
    'Find the most possible pinyin based on edit distance'
    # Prefer edit distance 0, then 1, then 2; otherwist default to word itself
    # edits0(word) 和 [word] 不是一样吗？？？
    candidates = (known(edits0(word)) or
                  known(edits1(word)) or
                  known(edits2(word)) or
                  [word])
    return max(candidates,key=PINYIN_COUNT.get)

In [48]:
def known(words):
    'Return the pinyin in our data'
    return {w for w in words if w in PINYIN_COUNT}

def edits0(word):
    'Return all strings that are zero edits away from word (i.e., just word itself).'
    return {word}

def edits2(word):
    'Return all strings that are two edits away from this pinyin.'
    return {e2 for e1 in edits1(word) for e2 in edits1(e1)}

In [16]:
alphabet = 'abcdefghijklmnopqrstuvwxyz'

def splits(word):
    'Return a list of all possible (first, rest) pairs that comprise pinyin.'
    return [(word[:i], word[i:])
           for i in range(len(word)+1)]

def edits1(word):
    'Return all strings that are one edit away from this pinyin.'
    pairs = splits(word)
    deletes = [a+b[1:] for (a,b) in pairs if b]
    transposes = [a+b[1]+b[0]+b[2:] for (a,b) in pairs if len(b) > 1]
    replaces = [a+c+b[1:] for (a,b) in pairs for c in alphabet if b]
    inserts = [a+c+b for (a,b) in pairs for c in alphabet]
    return set(deletes + transposes + replaces + inserts)

In [17]:
splits('pinyin')

[('', 'pinyin'),
 ('p', 'inyin'),
 ('pi', 'nyin'),
 ('pin', 'yin'),
 ('piny', 'in'),
 ('pinyi', 'n'),
 ('pinyin', '')]

In [18]:
print(edits0('pinyin'))

{'pinyin'}


In [19]:
print(edits1('pinyin'))

{'pinxin', 'pinyiin', 'pfnyin', 'pinymin', 'pinyion', 'pjnyin', 'pinpin', 'minyin', 'pinyie', 'pinyni', 'zinyin', 'psinyin', 'pinyiqn', 'rpinyin', 'pinysin', 'pinnin', 'pinyifn', 'zpinyin', 'vinyin', 'pinoyin', 'pinyind', 'jinyin', 'piiyin', 'phinyin', 'pinyins', 'pcinyin', 'linyin', 'dinyin', 'hinyin', 'pinyinj', 'pinjyin', 'pinjin', 'pinyibn', 'pinyinr', 'pinkin', 'pinyfn', 'pignyin', 'kinyin', 'pinyisn', 'pinyinc', 'pinyvn', 'pbnyin', 'pinzin', 'pinyjin', 'pinyinq', 'pinyib', 'piknyin', 'pijnyin', 'ponyin', 'pinyitn', 'cpinyin', 'tinyin', 'piniyn', 'pinynin', 'pinwyin', 'pinyxn', 'pxnyin', 'pinyxin', 'piniyin', 'pinyain', 'ppinyin', 'ypinyin', 'pinyix', 'pinyinb', 'pinydn', 'pibnyin', 'opinyin', 'pilnyin', 'pinyyin', 'pinyiwn', 'pinmin', 'ptinyin', 'pinybin', 'pinydin', 'pinyyn', 'pinyinf', 'pinypin', 'pinyijn', 'puinyin', 'pinyixn', 'pienyin', 'pinyin', 'pqinyin', 'picnyin', 'pinlyin', 'pinyia', 'pinyif', 'pinyim', 'winyin', 'hpinyin', 'pinyinz', 'piunyin', 'pigyin', 'pinyiyn', 'pi

# Test

In [20]:
correct('yin')

'yin'

In [21]:
correct('yign')

'ying'

In [22]:
correct('yinn')

'ying'

In [23]:
def correct_sequence_pinyin(text_pinyin):
    return ' '.join(map(correct, text_pinyin.split()))

In [24]:
correct_sequence_pinyin('zhe sih yi ge ce sho')

'zhe shi yi ge ce shi'

In [25]:
correct_sequence_pinyin('wo xiang shagn qinng hua da xue')

'wo xiang shang qing hua da xue'

# 思考题-homework？    
#### 如何在不带空格的时候完成自动修正？--> 如何完成拼音的分割？   
###### 提示：使用第一节课提到的语言模型!  n-gram

woyaoshangqinghua

w yaoshangqinghua

wo yaoshangqinghua

woyao shangqinghua

-> DP

In [136]:
# 把所有的可能性都切出来，看哪种的 可能性最高
# 对得到的每个结果，还要递归地切,直到每个结果都只有一个字母为止
# 是否还需要考虑几个词连起来 是否可能性最大？？？

# 首先假设任意位置分割，假设切出来的左右两部分语言概率已经算出来，那这个分割方法对应的概率就是左边乘以右边，
# 一个长度为 n 的字符串应该有 n 种切法，根据前面的概率选择出概率最大的一种切法。
# 以此递归，结束条件就是字符串不切割了视为一个词，把这个词尝试纠错然后获得概率
from functools import lru_cache

cuts = {}
@lru_cache(maxsize=2**10)
def cut_pinyin(inputs):
    inputs = inputs.strip()
    max_freq, max_cut = max(
        [(correct_a_word(inputs)[1], len(inputs))] + [(cut_pinyin(inputs[:i]) * cut_pinyin(inputs[i:]), i) for i in range(len(inputs)+1)],
        key=lambda x: x[0]
    )
    cuts[inputs] = max_cut
    
    return max_freq

In [137]:
def correct_a_word(word):
    prob = PINYIN_COUNT[word] / sum(PINYIN_COUNT.values())
    candidates0 = (word, prob, 0)
    
    if known(edits1(word)):
        word1 = max(known(edits1(word)), key=PINYIN_COUNT.get)
        prob1 = 1.0 / 2**15 * PINYIN_COUNT[word1]/sum(PINYIN_COUNT.values())
        candidates1 = (word1, prob1, 1)
        return max({candidates0, candidates1}, key = lambda x: x[1])
    else:
        return candidates0